In [1]:
library(tidyverse)
library(data.table)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [2]:
phe_file <- '/oak/stanford/groups/mrivas/projects/biomarkers/covariate_corrected/output/phenotypes/full.table.phe'

In [3]:
phe_file_cols <- fread(
    cmd=paste0('cat ', phe_file, '| awk "NR == 1"', '| tr "\t" "\n"'),
    head=FALSE
)

In [4]:
covar_list <- fread(
    'covar.list', head=FALSE
)

In [5]:
phe_file_cols %>% dim()

[1] 222   1

In [6]:
phe_file_cols %>% 
mutate(
    col_idx = row_number()
) %>%
head()

V1,col_idx
FID,1
IID,2
age,3
sex,4
Array,5
PC1,6


In [7]:
covar_list %>% dim()

[1] 127   1

In [8]:
covar_list %>% filter(
    str_detect(V1, '[*]', negate = TRUE)
) %>% dim()


[1] 112   1

In [9]:
covar_list %>% filter(
    str_detect(V1, '[*]', negate = TRUE)
) %>% left_join(
    phe_file_cols %>% 
    mutate(
        col_idx = row_number()
    ),
    on='V1'
) %>% 
select(
    col_idx
) %>% as.list() %>% paste0()

Joining, by = "V1"


[1] "3:114"

### the 1st order covariates are in columns 3-114. With FID and IID, we need to read cols 1-114

In [10]:
phe_df <- fread(
    cmd=paste0('cat ', phe_file, '| cut -f1-114'),
    head=TRUE
)

In [11]:
covar_list %>% filter(
    str_detect(V1, '[*]', negate = FALSE)
) %>% mutate(
    V2 = str_replace(V1, ' [*] ', '_')
) %>% mutate(
    V3 = paste(V2, V1, sep=' = ')
) %>% select(V3)


V3
age_age = age * age
age_sex = age * sex
sex_DrawTime = sex * DrawTime
age_DrawTime = age * DrawTime
sex_UrineSampleMinutes = sex * UrineSampleMinutes
age_UrineSampleMinutes = age * UrineSampleMinutes
DrawTime_DrawTime = DrawTime * DrawTime
FastingTime_FastingTime = FastingTime * FastingTime
UrineSampleMinutes_UrineSampleMinutes = UrineSampleMinutes * UrineSampleMinutes
DilutionFactorTimeZero_DrawTime = DilutionFactorTimeZero * DrawTime


In [12]:
phe_computed_df <- phe_df %>% mutate(
    age_age = age * age,
    age_sex = age * sex,
    sex_DrawTime = sex * DrawTime,
    age_DrawTime = age * DrawTime,
    sex_UrineSampleMinutes = sex * UrineSampleMinutes,
    age_UrineSampleMinutes = age * UrineSampleMinutes,
    DrawTime_DrawTime = DrawTime * DrawTime,
    FastingTime_FastingTime = FastingTime * FastingTime,
    UrineSampleMinutes_UrineSampleMinutes = UrineSampleMinutes * UrineSampleMinutes,
    DilutionFactorTimeZero_DrawTime = DilutionFactorTimeZero * DrawTime,
    DilutionFactorTimeZero_UrineSampleMinutes = DilutionFactorTimeZero * UrineSampleMinutes,
    sex_FastingTime = sex * FastingTime,
    age_FastingTime = age * FastingTime,
    age_TDI = age * TDI,
    sex_TDI = sex * TDI
) 

In [13]:
phe_computed_df %>% dim()

[1] 461984    129

In [14]:
phe_computed_df %>% 
fwrite(
    'biomarker_covar.tsv', sep='\t'
)

In [19]:
covar_list %>% 
mutate(
    V2 = str_replace(V1, ' [*] ', '_')
) %>% select(V2) %>%
fwrite(
    'biomarker_covar.colnames',
    col.names = FALSE
)


In [25]:
covar_list %>% mutate(
    beta = row_number()
) %>% 
# foo
right_join(
    data.frame(V1 = c('age', 'sex')),
    on='ID'
)

Joining, by = "V1"
Warning message:
“Column `V1` joining character vector and factor, coercing into character vector”

V1,beta
age,1
sex,2


In [26]:
!pwd

ERROR: Error in eval(expr, envir, enclos): object 'pwd' not found
